In [1]:
import pandas as pd
import re
import numpy as np
from DAL_Class_1 import DAL
import processing_functions
from datetime import datetime
from site_information_class import SiteDataProcessor
import tkinter as tk
from tkinter import simpledialog, scrolledtext 
from tkinter import ttk
from PIL import Image, ImageTk
from tkinter import messagebox
import math
import os
from Class_tkinter import SiteInformationApp  

Runs the downloader window

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = SiteInformationApp(root)
    root.mainloop()
    # Access the DataFrame after the main loop
    df_spill_hours = app.df_spill_hours
    df_rainfall_raw = app.df_rainfall_global
    df_raw_sump = app.df_raw_sump_global
    df_hour_agg_flow_meter_raw = app.df_hour_agg_flow_meter_global
    #df_raw_flow_meter = app.df_raw_flow_meter_global
    start_date_downloaded = app.start_date_global
    end_date_downloaded = app.end_date_global
    site_id_selected = app.site_id


Saves the downloaded dataframes into the /raw folder

In [ ]:
# Define file paths
file_paths = {
    #"df_spill_hours": "../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_spill_hours.xlsx",
    "df_rainfall_raw": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_rainfall_raw.xlsx",
    "df_raw_sump": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_raw_sump.xlsx",
    "df_hour_agg_flow_meter_raw": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_hour_agg_flow_meter_raw.xlsx",
    #"df_raw_flow_meter": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_raw_flow_meter.xlsx"
}

# Function to save dataframes to xlsx files
def save_dataframes(**kwargs):
    for name, df in kwargs.items():
        file_path = file_paths.get(name)
        if file_path:
            df.to_excel(file_path, index=False)
            print(f"{name} saved to {file_path}")

# Example usage:
# Assuming the dataframes are already defined and available as variables
# df_spill_hours, df_rainfall, df_raw_sump, df_hour_agg_flow_meter, df_raw_flow_meter

# Save the dataframes
save_dataframes(
    #df_spill_hours=df_spill_hours,
    df_rainfall_raw=df_rainfall_raw,
    df_raw_sump=df_raw_sump,
    df_hour_agg_flow_meter_raw=df_hour_agg_flow_meter_raw
    #df_raw_flow_meter=df_raw_flow_meter
)



Imports the downloaded data from excel workbooks - skip this if already in the environment from the DAL

In [42]:

import os
import pandas as pd

site_id_selected = 19505
start_date_downloaded = '2023-01-01'
end_date_downloaded = '2024-01-01'

file_paths = {
    #"df_spill_hours": "../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_spill_hours.xlsx",
    "df_rainfall_raw": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_rainfall_raw.xlsx",
    "df_raw_sump": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_raw_sump.xlsx",
    "df_hour_agg_flow_meter_raw": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_hour_agg_flow_meter_raw.xlsx",
    #"df_raw_flow_meter": f"../data/raw/site_id{site_id_selected}_{start_date_downloaded}_to_{end_date_downloaded}_df_raw_flow_meter.xlsx"
}

# Function to load dataframes from xlsx files if they exist
def load_dataframes():
    dataframes = {}
    for name, file_path in file_paths.items():
        if os.path.exists(file_path):
            dataframes[name] = pd.read_excel(file_path)
            print(f"{name} loaded from {file_path}")
        else:
            dataframes[name] = None
            print(f"{file_path} does not exist. {name} not loaded.")
    return dataframes

# Load the dataframes if they exist
loaded_dataframes = load_dataframes()

# Access the loaded dataframes
#df_spill_hours = loaded_dataframes["df_spill_hours"]

df_rainfall_raw = loaded_dataframes["df_rainfall_raw"]
df_raw_sump = loaded_dataframes["df_raw_sump"]
df_hour_agg_flow_meter_raw = loaded_dataframes["df_hour_agg_flow_meter_raw"]
#df_raw_flow_meter = loaded_dataframes["df_raw_flow_meter"]


df_rainfall_raw loaded from ../data/raw/site_id19505_2023-01-01_to_2024-01-01_df_rainfall_raw.xlsx
df_raw_sump loaded from ../data/raw/site_id19505_2023-01-01_to_2024-01-01_df_raw_sump.xlsx
df_hour_agg_flow_meter_raw loaded from ../data/raw/site_id19505_2023-01-01_to_2024-01-01_df_hour_agg_flow_meter_raw.xlsx


In [43]:
df_hour_agg_flow_meter_raw.head(10)

,Hour,DbAddr,count,meanEValue,stddev_EValue,Year,Month,Day
0,0,11726,60,38.30,1.725505,2023,1,1
1,1,11726,60,38.15,2.091776,2023,1,1
2,2,11726,60,38.54,1.841687,2023,1,1
3,3,11726,60,38.18,1.845763,2023,1,1
4,4,11726,60,38.71,1.728748,2023,1,1
5,5,11726,60,38.36,1.850873,2023,1,1
6,6,11726,60,38.72,1.652161,2023,1,1
7,7,11726,60,38.73,1.606117,2023,1,1
8,8,11726,60,38.40,1.963411,2023,1,1
9,9,11726,60,38.03,1.830906,2023,1,1


In [44]:

def transform_flow_meter_data(df_hour_agg_flow_meter):
    # Create the 'TimeGMT' column by combining Year, Month, Day, and Hour columns
    df_hour_agg_flow_meter['TimeGMT'] = pd.to_datetime(
        df_hour_agg_flow_meter['Year'].astype(str) + '-' +
        df_hour_agg_flow_meter['Month'].astype(str) + '-' +
        df_hour_agg_flow_meter['Day'].astype(str) + ' ' +
        df_hour_agg_flow_meter['Hour'].astype(str) + ':00:00'
    )
    
    # Sort the dataframe by 'TimeGMT'
    df_hour_agg_flow_meter_filtered = df_hour_agg_flow_meter.sort_values(by="TimeGMT")
    
    # Drop unnecessary columns
    df_hour_agg_flow_meter_filtered = df_hour_agg_flow_meter_filtered.drop(columns=['stddev_EValue', 'count', 'DbAddr'])
    
    return df_hour_agg_flow_meter_filtered

In [64]:
df_hour_agg_flow_meter = transform_flow_meter_data(df_hour_agg_flow_meter_raw) 

df_hour_agg_flow_meter.head()


,Hour,meanEValue,Year,Month,Day,TimeGMT
0,0,38.30,2023,1,1,2023-01-01 00:00:00
1,1,38.15,2023,1,1,2023-01-01 01:00:00
2,2,38.54,2023,1,1,2023-01-01 02:00:00
3,3,38.18,2023,1,1,2023-01-01 03:00:00
4,4,38.71,2023,1,1,2023-01-01 04:00:00


In [65]:
print(df_hour_agg_flow_meter.columns)

Index(['Hour', 'meanEValue', 'Year', 'Month', 'Day', 'TimeGMT'], dtype='object')


In [66]:

# Make a copy of the DataFrame
df_hour_agg_flow_meter_t = df_hour_agg_flow_meter.copy()

# Convert TimeGMT to datetime
df_hour_agg_flow_meter_t['TimeGMT'] = pd.to_datetime(df_hour_agg_flow_meter_t['TimeGMT'])

# Set TimeGMT as the index
df_hour_agg_flow_meter_t.set_index('TimeGMT', inplace=True)

# Ensure the index has a frequency set
df_hour_agg_flow_meter_t = df_hour_agg_flow_meter_t.asfreq('h')

# Check if there are enough observations
if len(df_hour_agg_flow_meter_t) < 48:
    print("Not enough data points for decomposition. Need at least 48 observations.")
else:
    # Perform time series decomposition
    result = seasonal_decompose(df_hour_agg_flow_meter_t['meanEValue'], model='additive', period=24)

    # Extract the trend component
    df_trend = result.trend.dropna()

    print(df_trend)

# Display the first few rows of the trend component
df_trend.head()

TimeGMT
2023-01-01 12:00:00    37.455000
2023-01-01 13:00:00    37.454792
2023-01-01 14:00:00    37.443542
2023-01-01 15:00:00    37.430417
2023-01-01 16:00:00    37.415625
                         ...    
2024-01-01 07:00:00    41.498750
2024-01-01 08:00:00    41.449375
2024-01-01 09:00:00    41.404792
2024-01-01 10:00:00    41.338125
2024-01-01 11:00:00    41.277083
Freq: h, Name: trend, Length: 8760, dtype: float64


TimeGMT
2023-01-01 12:00:00    37.455000
2023-01-01 13:00:00    37.454792
2023-01-01 14:00:00    37.443542
2023-01-01 15:00:00    37.430417
2023-01-01 16:00:00    37.415625
Freq: h, Name: trend, dtype: float64

In [67]:
# Make a copy of the DataFrame
df_hour_agg_flow_meter_t = df_hour_agg_flow_meter.copy()

# Convert TimeGMT to datetime
df_hour_agg_flow_meter_t['TimeGMT'] = pd.to_datetime(df_hour_agg_flow_meter_t['TimeGMT'])

# Set TimeGMT as the index
df_hour_agg_flow_meter_t.set_index('TimeGMT', inplace=True)

# Ensure the index has a frequency set
df_hour_agg_flow_meter_t = df_hour_agg_flow_meter_t.asfreq('h')

# Check if there are enough observations
if len(df_hour_agg_flow_meter_t) < 48:
    print("Not enough data points for decomposition. Need at least 48 observations.")
else:
    # Perform time series decomposition
    result = seasonal_decompose(df_hour_agg_flow_meter_t['meanEValue'], model='additive', period=24)

    # Extract the trend component
    df_trend = result.trend.dropna()

    # Create a new column 'flow_trend' in df_hour_agg_flow_meter_t
    df_hour_agg_flow_meter_t['flow_trend'] = df_trend

    # Fill missing values in 'flow_trend' with 'meanEValue' from the original DataFrame
    df_hour_agg_flow_meter_t['flow_trend'].fillna(df_hour_agg_flow_meter_t['meanEValue'], inplace=True)

    # Display the updated DataFrame
    print(df_hour_agg_flow_meter_t)

# Display the first few rows of the trend component
df_hour_agg_flow_meter_t.head()

                     Hour  meanEValue  Year  Month  Day  flow_trend
TimeGMT                                                            
2023-01-01 00:00:00     0       38.30  2023      1    1       38.30
2023-01-01 01:00:00     1       38.15  2023      1    1       38.15
2023-01-01 02:00:00     2       38.54  2023      1    1       38.54
2023-01-01 03:00:00     3       38.18  2023      1    1       38.18
2023-01-01 04:00:00     4       38.71  2023      1    1       38.71
...                   ...         ...   ...    ...  ...         ...
2024-01-01 19:00:00    19       39.67  2024      1    1       39.67
2024-01-01 20:00:00    20       40.92  2024      1    1       40.92
2024-01-01 21:00:00    21       39.25  2024      1    1       39.25
2024-01-01 22:00:00    22       40.06  2024      1    1       40.06
2024-01-01 23:00:00    23       40.18  2024      1    1       40.18

[8784 rows x 6 columns]


C:\Users\RMCGINT\AppData\Local\Temp\ipykernel_26524\3593859513.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_hour_agg_flow_meter_t['flow_trend'].fillna(df_hour_agg_flow_meter_t['meanEValue'], inplace=True)


,Hour,meanEValue,Year,Month,Day,flow_trend
TimeGMT,,,,,,
2023-01-01 00:00:00,0,38.30,2023,1,1,38.30
2023-01-01 01:00:00,1,38.15,2023,1,1,38.15
2023-01-01 02:00:00,2,38.54,2023,1,1,38.54
2023-01-01 03:00:00,3,38.18,2023,1,1,38.18
2023-01-01 04:00:00,4,38.71,2023,1,1,38.71


In [70]:
df_hour_agg_flow_meter.head()

,Hour,meanEValue,Year,Month,Day,TimeGMT
0,0,38.30,2023,1,1,2023-01-01 00:00:00
1,1,38.15,2023,1,1,2023-01-01 01:00:00
2,2,38.54,2023,1,1,2023-01-01 02:00:00
3,3,38.18,2023,1,1,2023-01-01 03:00:00
4,4,38.71,2023,1,1,2023-01-01 04:00:00


In [73]:
# Reset the index to revert 'TimeGMT' from being the index to a column
df_hour_agg_flow_meter_t.reset_index(inplace=True)

# Now 'TimeGMT' is a column again, and the index is back to default integers
print(df_hour_agg_flow_meter_t.head())

              TimeGMT  Hour  meanEValue  Year  Month  Day  flow_trend
0 2023-01-01 00:00:00     0       38.30  2023      1    1       38.30
1 2023-01-01 01:00:00     1       38.15  2023      1    1       38.15
2 2023-01-01 02:00:00     2       38.54  2023      1    1       38.54
3 2023-01-01 03:00:00     3       38.18  2023      1    1       38.18
4 2023-01-01 04:00:00     4       38.71  2023      1    1       38.71


This runs the time series viewer / rtk optimizer

In [76]:
df_sump_filtered = df_raw_sump
df_sump_filtered = df_sump_filtered.sort_values(by="TimeGMT")
df_rainfall= processing_functions.process_rainfall_data(df_rainfall_raw)
#df_hour_agg_flow_meter = processing_functions.transform_flow_meter_data(df_hour_agg_flow_meter_raw)


from Plotting_raw_data_class_d import PlotWindow


if __name__ == "__main__":
    import tkinter as tk

    root = tk.Tk()
    start_date_plot = '01-01-2023'
    end_date_plot = '01-01-2024'
    app2 = PlotWindow(root, start_date_plot, end_date_plot, df_raw_sump=df_sump_filtered, df_rainfall=df_rainfall, df_hour_agg_flow_meter=df_hour_agg_flow_meter_t, spill_level=100, sump_ylim=100, flow_ylim=100)
    root.mainloop()
    '''

    # Access RTK parameters and synthetic flow after the main loop ends
    R, T, K, df_synthetic_flow = app2.get_rtk_parameters_and_synthetic_flow()

    print(df_synthetic_flow.head())
    '''
    
    # Access RTK parameters and synthetic flow after the main loop ends
    R1, T1, K1, R2, T2, K2, df_synthetic_flow = app2.get_rtk_parameters_and_synthetic_flow()
    print(f"Retrieved RTK Parameters Set 1: R1 = {R1}, T1 = {T1}, K1 = {K1}")
    print(f"Retrieved RTK Parameters Set 2: R2 = {R2}, T2 = {T2}, K2 = {K2}")
    print(df_synthetic_flow.head())


c:\Users\RMCGINT\OneDrive - Wessex Water\Python\WWDS project\scripts\processing_functions.py:420: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_rainfall_hour_agg = df_rainfall_filtered.resample('h').sum(numeric_only=True) / 12


Retrieved RTK Parameters Set 1: R1 = None, T1 = None, K1 = None
Retrieved RTK Parameters Set 2: R2 = None, T2 = None, K2 = None


AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:

from processing_functions import process_spill_hours

df_spill_hours = processing_functions.process_spill_hours(df_spill_hours)


# Convert spill_hours to datetime
df_spill_hours['spill_hours'] = pd.to_datetime(df_spill_hours['spill_hours'])

# Extract year from spill_hours
df_spill_hours['year'] = df_spill_hours['spill_hours'].dt.year

# Group by year and spill_event_id and get the maximum spill_event_duration for each group
max_durations = df_spill_hours.groupby(['year', 'spill_event_id'])['spill_event_duration'].max().reset_index()

# Create a box plot for the distribution of the maximum spill_event_durations for each year
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.boxplot(x='year', y='spill_event_duration', data=max_durations)
plt.title('Distribution of Spill Event Durations by Year')
plt.xlabel('Year')
plt.ylabel('Maximum Spill Event Duration (hours)')



# Create an overall box plot for the distribution of the maximum spill_event_durations
plt.subplot(1, 2, 2)
sns.boxplot(y=max_durations['spill_event_duration'])
plt.title('Overall Distribution of Spill Event Durations')
plt.ylabel('Maximum Spill Event Duration (hours)')



plt.gca().set_aspect(0.075)

plt.tight_layout()
plt.show()


In [ ]:



site_id = 19505
# Calculate total spill hours in each Month for each Year
total_spill_hours_per_month_year = df_spill_hours.groupby(['Year', 'Month']).size().unstack(fill_value=0)

# Create a bar chart of total spill hours in each Month with different colors for each Year
fig, ax = plt.subplots(figsize=(10, 6))

# Plot polygons for each Year with high transparency and add a line for each Year
for year in total_spill_hours_per_month_year.index:
    ax.fill_between(total_spill_hours_per_month_year.columns, 0, total_spill_hours_per_month_year.loc[year], alpha=0.25, label=str(year))
    ax.plot(total_spill_hours_per_month_year.columns, total_spill_hours_per_month_year.loc[year], marker='o')

# Set the x-axis labels to be the Months
ax.set_xticks(total_spill_hours_per_month_year.columns)
ax.set_xticklabels(total_spill_hours_per_month_year.columns)

ax.set_xlabel('Month')
ax.set_ylabel('Total Spill Hours')
ax.set_title(f'Total Spill Hours in Each Month by Year for Site ID {site_id} using sump level {DB_Addr_sump_str}')
ax.legend(title='Year')
plt.grid(True)
plt.show()



In [17]:
df_hour_agg_flow_meter.head()

,Hour,meanEValue,Year,Month,Day,TimeGMT
0,0,38.30,2023,1,1,2023-01-01 00:00:00
1,1,38.15,2023,1,1,2023-01-01 01:00:00
2,2,38.54,2023,1,1,2023-01-01 02:00:00
3,3,38.18,2023,1,1,2023-01-01 03:00:00
4,4,38.71,2023,1,1,2023-01-01 04:00:00


In [25]:

import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Assuming df_hour_agg_flow_meter is already defined in your environment

# Strip any leading or trailing spaces from column names
df_hour_agg_flow_meter.columns = df_hour_agg_flow_meter.columns.str.strip()





# Perform time series decomposition
result = seasonal_decompose(df_hour_agg_flow_meter['meanEValue'], model='additive')

# Plot the decomposition results
result.plot()
plt.show()

ValueError: You must specify a period or x must be a pandas object with a PeriodIndex or a DatetimeIndex with a freq not set to None

In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt



# Define the timeframe you want to zone in on
start_date = '2023-03-01'
end_date = '2023-09-30'

# Filter the dataframe for the specified timeframe
df_filtered = df_hour_agg_flow_meter.loc[start_date:end_date]

# Perform time series decomposition on the filtered data
result = seasonal_decompose(df_filtered['meanEValue'], model='additive')

# Plot the decomposition results
result.plot()
plt.show()




In [ ]:
print(result.trend.dropna().head())